In [ ]:
import os.path as osp
from math import ceil, floor
import numpy as np

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.nn import DenseGCNConv as GCNConv, dense_diff_pool
from torch_geometric.nn import DenseSAGEConv
from torch_geometric.datasets import TUDataset
from tqdm import tqdm


In [ ]:
max_nodes = 150


class MyFilter(object):
    def __call__(self, data):
        return data.num_nodes <= max_nodes


dataset = TUDataset('data', name='PROTEINS', transform=T.ToDense(max_nodes),
                    pre_filter=MyFilter())
dataset = dataset.shuffle()
n = (len(dataset) + 9) // 10
test_dataset = dataset[:n]
val_dataset = dataset[n:2 * n]
train_dataset = dataset[2 * n:]
test_loader = DenseDataLoader(test_dataset, batch_size=20)
val_loader = DenseDataLoader(val_dataset, batch_size=20)
train_loader = DenseDataLoader(train_dataset, batch_size=20)


class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 normalize=False, lin=True):
        super().__init__()

        self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        if lin is True:
            self.lin = torch.nn.Linear(2 * hidden_channels + out_channels,
                                       out_channels)
        else:
            self.lin = None

    def bn(self, i, x):
        batch_size, num_nodes, num_channels = x.size()

        x = x.view(-1, num_channels)
        x = getattr(self, f'bn{i}')(x)
        x = x.view(batch_size, num_nodes, num_channels)
        return x

    def forward(self, x, adj, mask=None):
        batch_size, num_nodes, in_channels = x.size()

        x0 = x
        x1 = self.bn(1, self.conv1(x0, adj, mask).relu())
        x2 = self.bn(2, self.conv2(x1, adj, mask).relu())
        x3 = self.bn(3, self.conv3(x2, adj, mask).relu())

        x = torch.cat([x1, x2, x3], dim=-1)

        if self.lin is not None:
            x = self.lin(x).relu()

        return x


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()

        num_nodes = ceil(0.25 * max_nodes)
        self.gnn1_pool = GNN(dataset.num_features, 64, num_nodes)
        self.gnn1_embed = GNN(dataset.num_features, 64, 64, lin=False)

        num_nodes = ceil(0.25 * num_nodes)
        self.gnn2_pool = GNN(3 * 64, 64, num_nodes)
        self.gnn2_embed = GNN(3 * 64, 64, 64, lin=False)

        self.gnn3_embed = GNN(3 * 64, 64, 64, lin=False)

        self.lin1 = torch.nn.Linear(3 * 64, 64)
        self.lin2 = torch.nn.Linear(64, dataset.num_classes)

    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)

        x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)

        x, adj, l2, e2 = dense_diff_pool(x, adj, s)

        x = self.gnn3_embed(x, adj)

        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2


# Implement Graph Deconvolution Layer
class GDN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, x, adj, mask=None):

        pass


class HierarchicalGAE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        
        #----------------- Graph Convolution (Encoding layers) -----------------#
        num_nodes1 = ceil(0.25 * max_nodes)
        self.gnn1_pool = GNN(dataset.num_features, 64, num_nodes1)
        self.gnn1_embed = GNN(dataset.num_features, 64, 64, lin=False)

        num_nodes2 = ceil(0.25 * num_nodes1)
        self.gnn2_pool = GNN(3 * 64, 64, num_nodes2)
        self.gnn2_embed = GNN(3 * 64, 64, 64, lin=False)

        self.gnn3_embed = GNN(3 * 64, 64, 64, lin=False)


        #----------------- Graph Deconvolution (Decoding layers) -----------------#

        self.gnn1_upsample = GNN(64, 64, 3 * 64, num_nodes1)
        self.gnn1_embed = GNN(64, 64, 3 * 64,lin=False)
        
        self.gnn2_upsample = GNN(64, 64, 3 * 64, max_nodes)
        self.gnn2_embed = GNN(64, 64, 3 * 64,lin=False)

    def encode(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)

        # x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)
        x = s.t() @ x
        adj = s.t() @ adj @ s

        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)

        x = s.t() @ x
        adj = s.t() @ adj @ s
        
        x = self.gnn3_embed(x, adj)

        x = x.mean(dim=1)

        return x, adj
    
    def decode(self, x, adj, mask=None):
        s = self.gnn1_upsample(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)
        x, adj, _, _ = dense_diff_pool(x, adj, s, mask)
        
        s = self.gnn2_upsample(x, adj, mask)
        x = self.gnn2_embed(x, adj, mask)
        
        x, adj, _, _ = dense_diff_pool(x, adj, s, mask)

        return x, adj

    def forward(self, x, adj, mask=None):
        x, adj = self.encode(x, adj, mask)
        
        return  self.decode(x, adj)



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hgae = HierarchicalGAE().to(device)
optimizer = torch.optim.Adam(hgae.parameters(), lr=0.001)

hgae.train()
for data in train_loader:
    data = data.to(device)
    optimizer.zero_grad()
    output, _ = hgae(data.x, data.adj, data.mask)
    print(output)

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        loss_all += data.y.size(0) * float(loss)
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0

    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.adj, data.mask)[0].max(dim=1)[1]
        correct += int(pred.eq(data.y.view(-1)).sum())
    return correct / len(loader.dataset)

best_val_acc = test_acc = 0
for epoch in range(1, 151):
    train_loss = train(epoch)
    val_acc = test(val_loader)
    if val_acc > best_val_acc:
        test_acc = test(test_loader)
        best_val_acc = val_acc
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
import networkx as nx
import torch_geometric

for data in train_loader:
    example = data.get_example(0)
    print(example.edge_index)
    g = torch_geometric.utils.to_networkx(example)
    nx.draw(g)
    break
